In [313]:
import pandas as pd
import numpy as np
import json
import math

In [383]:
amostra_estruturacao_file = "../dados/amostras_estruturacao/6.csv"

In [384]:
items_df = pd.read_csv(amostra_estruturacao_file, sep=',')

In [385]:
items_df

,id,descricao_original,,palavras_original,palavras_novo,unidades_medida_original,unidades_medida_novo,números_original,números_novo,cores_original,cores_novo,materiais_original,materiais_novo,tamanho_original,tamanho_novo,tipo_quantidade_original,tipo_quantidade_novo
0,1,001626 MELANCIA-,,melancia,#,NaN,#,1626,#,NaN,#,NaN,#,NaN,#,NaN,#
1,2,"Esparadrapo impermeável, com boa aderência, hi...",,"esparadrapo, impermeavel, bom, aderencia, hipo...","esparadrapo, impermeavel, bom, aderencia, hipo...","cm, m",#,"10, 4, 2005","10,4,5",branco,#,NaN,"algodao, tecido",NaN,#,NaN,rolo
2,3,REGUA 50 CM.,,regua,#,cm,#,50,#,NaN,#,NaN,#,NaN,#,NaN,#
3,4,"ENVELOPE TIPO SACO, SEM TIMBRE, SEM JANELA, EM...",,"envelope, saco, timbre, janela, papel, kraft, ...","envelope, saco, timbre, janela, kraft, natural...","g, m2, mm",#,"80, 180, 240",#,NaN,#,NaN,papel,NaN,#,NaN,#
4,5,PAPEL PRESENTE,,"papel, presente",presente,NaN,#,NaN,#,NaN,#,NaN,papel,NaN,#,NaN,#
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,146,BRACKETS ROTH SLOT22 2PM.I.ESQ,,"brackets, roth, slot22, pm, esq",#,NaN,#,2,#,NaN,#,NaN,#,NaN,#,NaN,#
146,147,CX COL P/MAT PERF CORT 7 LT,,"col, mat, perf, cort, lt","col, mat, perf, cort",lt,#,7,#,NaN,#,NaN,#,NaN,#,cx,#
147,148,US ORG E ESTR SUPERF (REG ING),,"org, estr, superf, reg, ing",#,NaN,#,NaN,#,NaN,#,NaN,#,NaN,#,NaN,#
148,149,NIPLE GALVANIZADO 2 - NBR5580M,,"niple, galvanizar, nbr",#,m,#,"2, 5580",#,NaN,#,NaN,#,NaN,#,NaN,#


In [386]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        150 non-null    int64 
 1   descricao_original        150 non-null    object
 2                             150 non-null    object
 3   palavras_original         150 non-null    object
 4   palavras_novo             150 non-null    object
 5   unidades_medida_original  72 non-null     object
 6   unidades_medida_novo      149 non-null    object
 7   números_original          115 non-null    object
 8   números_novo              150 non-null    object
 9   cores_original            19 non-null     object
 10  cores_novo                150 non-null    object
 11  materiais_original        22 non-null     object
 12  materiais_novo            150 non-null    object
 13  tamanho_original          5 non-null      object
 14  tamanho_novo              

In [387]:
items = items_df.values.tolist()

In [388]:
json_records = json.loads(items_df.to_json(orient ='records'))

In [389]:
palavras_added = set()
unidades_medida_added = set()
numeros_added = set()
cores_added = set()
materiais_added = set()
tamanho_added = set()
tipo_quantidade_added = set()

palavras_removed = set()
unidades_medida_removed = set()
numeros_removed = set()
cores_removed = set()
materiais_removed = set()
tamanho_removed = set()
tipo_quantidade_removed = set()

In [390]:
def get_words(words_original, words_novo):
    if str(words_original) == 'nan' or str(words_original) == 'None':
        words_original_set = set()
    else:
        words_original = words_original.split(',')
        words_original = [word.strip(' ').strip('\n') for word in words_original]
        words_original_set = set(words_original)
    
    if str(words_novo) == 'nan' or str(words_novo) == 'None':
        words_novo_set = set()
    elif '#' in words_novo:
        words_novo_set = words_original_set
    else:
        words_novo = words_novo.split(',')
        words_novo = [word.strip(' ').strip('\n') for word in words_novo]
        words_novo_set = set(words_novo)

    return words_original_set, words_novo_set

In [391]:
for item in json_records:
    palavras_original, palavras_novo = get_words(item["palavras_original"], item["palavras_novo"])
    palavras_added.update(list(palavras_novo - palavras_original))
    palavras_removed.update(list(palavras_original - palavras_novo))

    unidades_medida_original, unidades_medida_novo = get_words(item["unidades_medida_original"], item["unidades_medida_novo"])
    unidades_medida_added.update(list(unidades_medida_novo - unidades_medida_original))
    unidades_medida_removed.update(list(unidades_medida_original - unidades_medida_novo))
    
    numeros_original, numeros_novo = get_words(item["números_original"], item["números_novo"])
    numeros_added.update(list(numeros_novo - numeros_original))
    numeros_removed.update(list(numeros_original - numeros_novo))
    
    cores_original, cores_novo = get_words(item["cores_original"], item["cores_novo"])
    cores_added.update(list(cores_novo - cores_original))
    cores_removed.update(list(cores_original - cores_novo))
    
    materiais_original, materiais_novo = get_words(item["materiais_original"], item["materiais_novo"])
    materiais_added.update(list(materiais_novo - materiais_original))
    materiais_removed.update(list(materiais_original - materiais_novo))

    tamanho_original, tamanho_novo = get_words(item["tamanho_original"], item["tamanho_novo"])
    tamanho_added.update(list(tamanho_novo - tamanho_original))
    tamanho_removed.update(list(tamanho_original - tamanho_novo))

    tipo_quantidade_original, tipo_quantidade_novo = get_words(item["tipo_quantidade_original"], item["tipo_quantidade_novo"])
    tipo_quantidade_added.update(list(tipo_quantidade_novo - tipo_quantidade_original))
    tipo_quantidade_removed.update(list(tipo_quantidade_original - tipo_quantidade_novo))

In [392]:
for word in palavras_added:
    print(word.strip(' '))

galvanizada
prazo
qualidade
prima
posto
contendo


In [393]:
for word in palavras_removed:
    print(word.strip(' '))


feltro
galvanizar
primo
algodao
por
seis
dois
transparente
prazer
noventa
lt
tecido
mt
papel
rolo
no25
trinta
borracha
silicone
contender


In [394]:
for word in unidades_medida_added:
    print(word.strip(' ').strip('\n'))

mt


In [395]:
for word in unidades_medida_removed:
    print(word.strip(' '))

v


In [396]:
for word in numeros_added:
    print(word.strip(' '))

5
12
250
75
21
1000
100
2000
3510


In [397]:
for word in numeros_removed:
    print(word.strip(' '))

2005
572
1
0100
9
000
2
2000
0
0250


In [398]:
for word in cores_added:
    print(word.strip(' '))

transparente


In [399]:
for word in cores_removed:
    print(word.strip(' '))

In [400]:
for word in materiais_added:
    print(word.strip(' '))

tecido
feltro
acrilico
resina
algodao
papel
gesso


In [401]:
for word in materiais_removed:
    print(word.strip(' '))

In [402]:
for word in tamanho_added:
    print(word.strip(' '))

In [403]:
for word in tamanho_removed:
    print(word.strip(' '))

pequeno


In [381]:
for word in tipo_quantidade_added:
    print(word.strip(' '))

rolo


In [382]:
for word in tipo_quantidade_removed:
    print(word.strip(' '))

In [2]:
unidades_medida_new = {'kva', 'mhz', 'mes', 'grs', 'gota', 'tb', 'nm', 'oc', 'gb', 'ghz', 'rpm', 'lpm', 'minuto', 'segundo', 'abs', 'wats', 'mb', 'abs'
, 'mb', 'mhz', 'mes', 'gb', 'ghz', 'gr', 'segundo', 'nm', 'minuto', 'rpm', 'lpm', 'hz', 'mA', 'segundos', 'lt', 'wts', 'hz', 'mmol', 'Ma', 'mol'
, 'dl', 'lt', '%', 'mt', 'ag', 'mes', 'cir', 'lts', 'gb', 'lt', 'ano', 'grs', 'tb', '%', 'hp', 'min', 'nm', 'quilo', 'lts', 'gb', 'btua', 'tb', 'lt', 'nm'}

In [5]:
quantidade_new = {'ampola', 'cartela', 'frasco', 'frasco', 'bloco', 'po', 'bisnaga', 'injetavel', 'ampola', 'comprimido', 'frs', 'cp', 'bisnaga'
, 'frasco', 'ampola', 'comprimido', 'cp', 'rolo', 'und', 'frasco', 'kit', 'seringa', 'comprimido', 'ampola', 'frasco', 'und', 'ampola', 'comprimido'}

In [7]:
materiais_new = {'espuma', 'tecido', 'acrilico', 'celulose', 'resina', 'fibra', 'metalico', 'nylon', 'gesso', 'acrilico', 'resina', 'metalico'
, 'cromocobalto', 'emborrachar', 'elastico', 'nylon', 'gesso', 'eucalipto', 'poliestireno', 'cortica', 'polimero', 'polietileno', 'alum', 'tecido'
, 'couro', 'acrilico', 'resina', 'polipropileno', 'fibra', 'eucalipto', 'poliestireno', 'cortica', 'metalico', 'polimero', 'polietileno', 'mdf', 'alum'
,'tecido', 'couro' , 'madeira', 'acrilico', 'carbono', 'polipropileno', 'tecido', 'acrilico', 'lona', 'poliamida', 'metalico', 'poliester', 'nylon', 'tecido'
, 'feltro', 'acrilico', 'resina', 'gesso', 'poliester', 'resina', 'tungstenio', 'crepe', 'carbono', 'palha', 'cart', 'poliester', 'cera', 'nylon', 'ferro'
, 'metalico', 'aco', 'policarbonato', 'plastico', 'polipropileno', 'aluminio', 'viscose', 'policarbonato', 'sucupira', 'carbono', 'poliester' , 'polipropileno'
, 'nylon', 'angelim'}

In [3]:
len(unidades_medida_new)

37

In [8]:
for word in materiais_new:
    print(word)

espuma
mdf
aluminio
cortica
couro
palha
celulose
polipropileno
polimero
aco
ferro
eucalipto
fibra
crepe
emborrachar
poliamida
sucupira
lona
gesso
tecido
plastico
madeira
viscose
nylon
alum
acrilico
carbono
feltro
poliestireno
tungstenio
angelim
poliester
policarbonato
metalico
cart
elastico
resina
cera
cromocobalto
polietileno
